## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps
import ipywidgets as widgets

# Import API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Salalah,OM,17.0151,54.0924,82.22,28,44,10.54,scattered clouds
1,1,Jamestown,US,42.0970,-79.2353,33.39,65,90,5.99,light snow
2,2,Casino,AU,-28.8667,153.0500,75.56,76,75,6.80,broken clouds
3,3,Puerto Ayora,EC,-0.7393,-90.3518,67.93,76,25,6.38,scattered clouds
4,4,Qaanaaq,GL,77.4840,-69.3632,-7.73,88,96,7.31,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
min_temp

75.0

In [32]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & \
                                           (city_data_df["Max Temp"]<= max_temp)]    
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Salalah,OM,17.0151,54.0924,82.22,28,44,10.54,scattered clouds
2,2,Casino,AU,-28.8667,153.0500,75.56,76,75,6.80,broken clouds
5,5,Alofi,NU,-19.0595,-169.9187,79.30,75,74,12.55,broken clouds
7,7,Airai,TL,-8.9266,125.4092,76.51,52,86,2.13,overcast clouds
9,9,Kapaa,US,22.0752,-159.3190,79.12,78,90,1.01,light rain
11,11,Trincomalee,LK,8.5711,81.2335,82.04,76,93,6.38,overcast clouds
14,14,Hilo,US,19.7297,-155.0900,76.28,93,90,0.00,moderate rain
15,15,Jambi,ID,-1.6000,103.6167,89.06,62,99,1.16,overcast clouds
17,17,Grand Gaube,MU,-20.0064,57.6608,82.38,61,7,5.99,clear sky
23,23,Faanui,PF,-16.4833,-151.7500,81.23,75,100,10.80,overcast clouds


In [35]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [38]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df= preferred_cities_df.dropna()
clean_df.head(5)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Salalah,OM,17.0151,54.0924,82.22,28,44,10.54,scattered clouds
2,2,Casino,AU,-28.8667,153.0500,75.56,76,75,6.80,broken clouds
5,5,Alofi,NU,-19.0595,-169.9187,79.30,75,74,12.55,broken clouds
7,7,Airai,TL,-8.9266,125.4092,76.51,52,86,2.13,overcast clouds
9,9,Kapaa,US,22.0752,-159.3190,79.12,78,90,1.01,light rain


In [39]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Salalah,OM,82.22,scattered clouds,17.0151,54.0924,
2,Casino,AU,75.56,broken clouds,-28.8667,153.0500,
5,Alofi,NU,79.30,broken clouds,-19.0595,-169.9187,
7,Airai,TL,76.51,overcast clouds,-8.9266,125.4092,
9,Kapaa,US,79.12,light rain,22.0752,-159.3190,
11,Trincomalee,LK,82.04,overcast clouds,8.5711,81.2335,
14,Hilo,US,76.28,moderate rain,19.7297,-155.0900,
15,Jambi,ID,89.06,overcast clouds,-1.6000,103.6167,
17,Grand Gaube,MU,82.38,clear sky,-20.0064,57.6608,
23,Faanui,PF,81.23,overcast clouds,-16.4833,-151.7500,


In [76]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        #print(hotels["results"][0]["name"])
       
    except (IndexError):
        #print("Hotel not found... skipping.")
         hotel_df["Hotel Name"].replace('', np.nan)
   #clean_hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
            
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")            

-----------------------------
Data Retrieval Complete      
-----------------------------


In [87]:
# 7. Drop the rows where there is no Hotel Name.
#hotel_df_filter = hotel_df[(hotel_df["Hotel Name"]=="")]
#hotel_df_filter
clean_hotel_df = hotel_df.dropna()
#clean_hotel_df.isnull().sum()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Salalah,OM,82.22,scattered clouds,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
2,Casino,AU,75.56,broken clouds,-28.8667,153.0500,Squatters Homestead Motel
5,Alofi,NU,79.30,broken clouds,-19.0595,-169.9187,Taloa Heights
7,Airai,TL,76.51,overcast clouds,-8.9266,125.4092,Juvinal Martin
9,Kapaa,US,79.12,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
...,...,...,...,...,...,...,...
679,Tevaitoa,PF,81.00,overcast clouds,-16.7833,-151.5000,Hôtel Raiatea Lodge
684,Trairi,BR,79.97,overcast clouds,-3.2778,-39.2689,janelas do mar
685,Bathsheba,BB,79.41,scattered clouds,13.2167,-59.5167,Atlantis Hotel
686,Mwanza,TZ,75.00,light rain,-2.5167,32.9000,Gold Crest Hotel


In [88]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [8]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [9]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))